<h1> Desafio - Parte 2 </h1>
<h3> By: Luan Carlos Klein </h3>

<h2> Importação das bibliotecas </h2>

In [1]:
import os
import pandas as pd
import numpy as np
import ast

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

from nltk.sentiment import SentimentIntensityAnalyzer

from sklearn.cluster import DBSCAN

from sklearn import preprocessing

from sklearn.model_selection import train_test_split

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LinearRegression

from sklearn.neural_network import MLPRegressor

<h2>Definições iniciais</h2>

In [2]:
reviews_train_path = os.path.join("data/reviewsTrainToronto.csv")
x_train = os.path.join("data/X_trainToronto.csv")
df_reviews = pd.read_csv(reviews_train_path)
df = pd.read_csv(x_train)

<h2> Preparação dos dados </h2>

<b> Criação de Features </b>

<h3> Feature 1: Sequencias Frequentes nas Categorias </h3>

Encontra sequencias frequencias nas categorias (relacionadas a destaque)

In [3]:
#######################################################################################################
## Função que recebe um dataframe e uma lista de regras relacionada as categorias. E adiciona nesse dataset, cada uma dessas regras
## como True ou False, se o estabelecimento tiver ou não a sequencia
def add_feature_categories(df, rules):
    df_original = df.copy()
    def check_attr(cats, components):
        try:
            for i in components:
                if i not in cats:
                    return False
            return True
        except:
            return False
    for i in rules:
        df_original[str(i)] = df_original['categories'].apply(lambda x: check_attr(x, i))
    return df_original

#######################################################################################################
## Função que dado um dataframe, encontra quais são as as sequencias de categorais que implicam em ser ou não destaque
def find_features_categories(df):
    df_original = df
    lista_categorias = df['categories']
    lista_destaque = df['destaque']
    transactions = []
    cont = 0
    for i in lista_categorias:
        aux = str(i).split(', ')
        if lista_destaque[cont] == 0:
            aux.append("Não")
        else:
            aux.append("Sim")
        transactions.append(aux)
        cont += 1
    ##Transforma as transações em um dataframe
    te = TransactionEncoder()
    encoded = te.fit(transactions).transform(transactions)
    df = pd.DataFrame(encoded, columns=te.columns_)
    #Calcula os conjuntos de itens (itemsets) com suporte >= 0,
    frequent_itemsets = apriori(df, min_support=0.05, use_colnames=True)
    #Regras com o valor de confiança mínimo escolhido
    dfRegras= association_rules(frequent_itemsets, metric='confidence', min_threshold=0.65)
    dfRegras = dfRegras[ (dfRegras['consequents'] == {"Não"}) | (dfRegras['consequents'] == {"Sim"})]
    
    itens = []
    for i in dfRegras['antecedents']:
        aux = list(i)
        #if len(aux) == 1:
        itens.append(aux[0])
    return itens

<h3> Feature 2: Sequencias Frequentes nos atributos </h3>

In [4]:
# Lista com todos os atributos que há no dataset para o treino 
attr = ['BusinessParking_garage',
        'BusinessParking_street', 'BusinessParking_validated',
        'BusinessParking_lot', 'BusinessParking_valet', 'BikeParking',
        'RestaurantsPriceRange2', 'ByAppointmentOnly', 'GoodForKids',
        'WheelchairAccessible', 'RestaurantsReservations', 'RestaurantsAttire',
        'WiFi', 'RestaurantsGoodForGroups', 'Alcohol', 'Caters',
        'RestaurantsDelivery', 'NoiseLevel', 'OutdoorSeating',
        'GoodForMeal_dessert', 'GoodForMeal_latenight', 'GoodForMeal_lunch',
        'GoodForMeal_dinner', 'GoodForMeal_brunch', 'GoodForMeal_breakfast',
        'RestaurantsTakeOut', 'HasTV', 'Ambience_romantic', 'Ambience_intimate',
        'Ambience_classy', 'Ambience_hipster', 'Ambience_divey',
        'Ambience_touristy', 'Ambience_trendy', 'Ambience_upscale',
        'Ambience_casual', 'RestaurantsTableService', 'HappyHour',
        'BestNights_monday', 'BestNights_tuesday', 'BestNights_friday',
        'BestNights_wednesday', 'BestNights_thursday', 'BestNights_sunday',
        'BestNights_saturday', 'Smoking', 'GoodForDancing', 'Music_dj',
        'Music_background_music', 'Music_jukebox', 'Music_live', 'Music_video',
        'Music_karaoke', 'Music_no_music', 'CoatCheck', 'DogsAllowed',
        'DriveThru', 'AcceptsInsurance', 'BusinessAcceptsCreditCards',
        'DietaryRestrictions_dairy-free', 'DietaryRestrictions_gluten-free',
        'DietaryRestrictions_vegan', 'DietaryRestrictions_kosher',
        'DietaryRestrictions_halal', 'DietaryRestrictions_soy-free',
        'DietaryRestrictions_vegetarian', 'HairSpecializesIn_perms',
        'HairSpecializesIn_coloring', 'HairSpecializesIn_extensions',
        'HairSpecializesIn_curly', 'HairSpecializesIn_kids',
        'HairSpecializesIn_straightperms', 'HairSpecializesIn_africanamerican',
        'HairSpecializesIn_asian', 'BusinessAcceptsBitcoin', 'AgesAllowed', 'destaque']

#######################################################################################################
###### Função que dada o um datafrma com atributos e as regras, cria colunas de acordo com as regras
def add_feature_attributes(df_attributes, rules):
    def put_value(df, feature_name, components):
        size = len(df.index)
        df[feature_name] = True
        cont = 0
        while cont < size:
            val = True
            for i in components:
                aux = i.split("?")
                if df.loc[cont, aux[0]] != aux[1]:
                    val =  False
                    break
            df.loc[cont, feature_name] = val
            cont += 1
        return df.copy()

    def add_attributes(df_features_in):
        df_features = df_features_in
        itens = []
        for i in rules:
            aux = list(i)
            feature_name = str(aux)
            df_features = put_value(df_features, feature_name, aux)
            itens.append(feature_name)
        return df_features
    
    df_prep = add_attributes(df_attributes)
    attr_drop = attr[:len(attr)-1] ## Tira o destaque da lista de atibutos
    df_prep =  df_prep.drop(attr_drop, axis=1)
    return df_prep
        
########################################################################
## Dado um dataframe, coloca todos os atributos como colunas
def gerate_attributes_df(df_original):
    df = df_original.copy()

    ######################################################################
    ## Função que separa os atributos em colunas
    def separe_attributes(df):
        df_x_train = df
        ##### PARTE 1 - SEPARAÇÃO DE TODOS OS ATRIBUTOS PRESENTES NOS DADOS ############################
        ## Pega a lista de todos os atributos
        attrs = df_x_train['attributes']

        ## Variaveis para armazenar as chaves e uma lista dos valores possiveis
        values = {}

        ## Percorre a lista dos atributos
        for i in attrs:
            ## Se for nan (nao tiver atributo), volta pro topo
            if str(i) == "nan":
                continue
            else:
                ## Transforma o objeto em um dicionário
                aux = ast.literal_eval(i)

                ## Percorre Esse dicionário
                for j in aux:
                    ## Transforma o valor (se der, vai transformar para um dicionário também)
                    aux_2 = ast.literal_eval(aux[j])
                    ## Se tiver valor nulo, apenas pula e volta para o inicio do loop
                    if type(aux_2) == type(None):
                        continue
                    ## Verifica se a chave que está percorrendo já foi identificada
                    if j in values:
                        ## Se for um dicionário dentro
                        if type(aux_2) is dict:
                            ## Pecorre esse dicionário
                            for k in aux_2:
                                ## Verifica se a chave já foi identificada
                                ## Senão foi, adiciona
                                if k not in values[j]:
                                    values[j][k] = ""
                                    continue

                    ############## Caso a chave não tenha sido identificada
                    else:
                        ## Verfica se é um dicionário ou um valor 'normal'
                        if type(aux_2) is dict:
                            ## Cria uma variavel para colocar internamente na armazenagem dos valores
                            dict_interno = {}
                            ## Percorre esse dicionário interno
                            for k in aux_2:
                                ## Preenche o dicionário interno
                                dict_interno[k] = ""
                            ## Faz o dicionário externo receber o dicionário interno criado
                            values[str(j)] = dict_interno
                        ## Caso seja um valor normal
                        else:
                            ## Adiciona nas variaveis, uma lista com aquele valor
                            values[str(j)] = ""
        ####################################################################

        ##### PARTE 2 - CRIANDO COLUNAS PARA CADA ATRIBUTOS ###########
        ## Se caso algum estabelecimento não tiver os dados, será colocado como FALSO por padrão

        ## Função que recebe um dicionário (x), uma chave (i), e uma chave interna (k)
        def get_value(x, i, k = False):
            ## Se x for nulo, então retorna False
            if str(x) == 'nan':
                return False

            ## Se não houver chave interna (ou seja, x[i] não é um dicionário)
            if not k:
                try:
                    ## Transforma o X em um dicionario e retorna o valor de x[i]
                    dic = ast.literal_eval(x)
                    return dic[i]
                except Exception as e:
                    return False
            ## Caso x[i] for um dicionário
            else:
                ## Tenta transforma e retorna o valor de x[i][k]
                try:
                    dic = ast.literal_eval(x)
                    dic = ast.literal_eval(dic[i])
                    return dic[k]
                except Exception as e:
                    return False

        ## Percorre todas as chaves encontradas anteriormente        
        for i in values.keys():
            ## Verifica se os dados são de um dicionário
            if type(values[i]) == dict:
                ## Percorre o dicionário, se for o caso, e chama a função para retornar o valor
                for k in values[i].keys():
                     df_x_train[str(i) + "_" + str(k)] = df_x_train["attributes"].apply(lambda x: get_value(x, i, k))
            ## Cria a coluna com o atributo e chama a função para retornar o valor adequado
            else:    
                df_x_train[i] = df_x_train["attributes"].apply(lambda x: get_value(x, i))
        ############################### PARTE 3 - Garantir que todos os atributos foram colocados - Se não tiver, coloca tudo como falso ####
        for i in attr:
            if i not in df.columns:
                df_x_train[i] = False

        return df_x_train
    
    df_attributes = separe_attributes(df)
    return df_attributes

###########################################################################################################
## Função que dado um dataframe, encontra quais são as as sequencias de atributos que implicam em ser ou não destaque
def find_feature_attributes(df):
    attrs = df['attributes']
    lista_destaque = df['destaque']
    transactions = []
    cont = 0
    for i in attrs:
        trans_unit = []
        try:
            aux = ast.literal_eval(i)
            for j in aux.keys():
                aux2 = ast.literal_eval(aux[j])
                if type(aux2) == dict:
                    pass
                else:
                    trans_unit.append(j+"?"+aux[j])#+"_"+aux[j]
        except:
            trans_unit = []
        finally:
            if lista_destaque[cont] == 0:
                trans_unit.append("Não")
            else:
                trans_unit.append("Sim")
            transactions.append(trans_unit)
        cont += 1

    ##Transforma as transações em um dataframe
    te = TransactionEncoder()
    encoded = te.fit(transactions).transform(transactions)
    df = pd.DataFrame(encoded, columns=te.columns_)
    #Calcula os conjuntos de itens (itemsets) com suporte >= 0,
    frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)

    #Regras com o valor de confiança mínimo escolhido
    dfRegras= association_rules(frequent_itemsets, metric='confidence', min_threshold=0.85)
    dfRegras = dfRegras[ (dfRegras['consequents'] == {"Não"}) | (dfRegras['consequents'] == {"Sim"})]
    
    return dfRegras['antecedents'] 

<h3> Feature 3 - Sentimentos das reviews </h3>

In [5]:
###########################################################################################################
## Função que adiciona uma feature sobre o humor dos reviews. Faz a média das reviews sobre cada estabelecimento
def add_feature_sentiment_reviews(df_reviews, df):
    sent = SentimentIntensityAnalyzer()
    df_reviews_relevant = df_reviews.copy()
    df_reviews_relevant['sentiment'] = df_reviews_relevant['text'].apply(lambda x: sent.polarity_scores(x)['compound'])
    df_reviews_relevant = df_reviews_relevant.groupby(['business_id']).mean().reset_index()
    df_reviews_relevant = df_reviews_relevant[['business_id', 'sentiment']]
    df2 = df.merge(df_reviews_relevant, left_on='business_id', right_on='business_id', how='left')
    df2['sentiment'] = df2['sentiment'].fillna(0)
    return df2


<h3> Feature 4 - Proximidade de Lugares Destaques - Não destaques </h3>

In [6]:
###########################################################################################################
## Função que cria dois modelos de clusterização com o DBSCAN, um para destaques e outro para não destaques, 
## e faz a classificação do dateset de acordo com a localização
def add_features_places_train(df):
    df_use = df.copy()
    ids = df_use['business_id']
    
    df_pop = df_use[df_use['destaque'] == 1]
    ids_pop = df_pop['business_id']
    df_no_pop = df_use[df_use['destaque'] == 0]
    ids_no_pop = df_pop['business_id']

    df_pop = df_pop[['latitude', 'longitude']]
    df_no_pop = df_no_pop[['latitude', 'longitude']]
    
    dbscanConfig_pop = DBSCAN(eps=0.001, min_samples=40)
    dbscanConfig_no_pop = DBSCAN(eps=0.0008, min_samples=50)
    
    dbscanResults_pop = dbscanConfig_pop.fit(df_pop)
    dbscanResults_no_pop = dbscanConfig_no_pop.fit(df_no_pop)
    
    list_pop = list(zip(ids_pop, dbscanResults_pop.labels_))
    df_pop_result = pd.DataFrame(list_pop, columns=['business_id', 'pop_zone'])
    
    list_no_pop = list(zip(ids_no_pop, dbscanResults_no_pop.labels_))
    df_no_pop_result = pd.DataFrame(list_pop, columns=['business_id', 'no_pop_zone'])
    
    df = df.merge(df_pop_result, left_on='business_id', right_on='business_id', how='left')
    df = df.merge(df_no_pop_result, left_on='business_id', right_on='business_id', how='left')
    
    df['pop_zone'] = df['pop_zone'].fillna(-1)
    df['no_pop_zone'] = df['no_pop_zone'].fillna(-1)
    
    return df, dbscanResults_pop, dbscanResults_no_pop

###########################################################################################################
## Função que dado um novo dataframe, classifica a localização de acordo com os modelos treinados
def add_features_places_test(df, model_pop, model_no_pop):
    df_use = df.copy()
    ids = df_use['business_id']
    df_use = df_use[['latitude', 'longitude']]
    ## Fonte: https://stackoverflow.com/questions/27822752/scikit-learn-predicting-new-points-with-dbscan
    def dbscan_predict(model, X):
        nr_samples = X.shape[0]
        y_new = np.ones(shape=nr_samples, dtype=int) * -1
        for i in range(nr_samples):
            diff = model.components_ - list(X.iloc[i, :])
            dist = np.linalg.norm(diff, axis=1)
            shortest_dist_idx = np.argmin(dist)
            if dist[shortest_dist_idx] < model.eps:
                y_new[i] = model.labels_[model.core_sample_indices_[shortest_dist_idx]]
        return y_new
    pred_zone_pop = dbscan_predict(model_pop, df_use)
    pred_zone_no_pop = dbscan_predict(model_no_pop, df_use)
    df['pop_zone'] = pred_zone_pop
    df['no_pop_zone'] = pred_zone_no_pop
    return df

<h3> Preparação dos dados para os modelos </h3>

In [7]:
## Chama os métodos que irão definir os atributos
features_categories = find_features_categories(df)
features_attr = find_feature_attributes(df)
dbscanResults_pop = None
dbscanResults_no_pop = None

###########################################################################################################
## Função que recebe os dataframes e realiza o pré-processamento dos dados, tanto para treino como para teste
## retorna um dataframe pronto para ser usado nos modelos
def pre_processor(df, df_reviews, train=True):
    df_use = df.copy()
    print("Adding Features - Attributes")
    df_attributes = gerate_attributes_df(df_use)
    df_use = add_feature_attributes(df_attributes, features_attr)
    print("Adding Features - Categories")
    df_use = add_feature_categories(df_use, features_categories)
    print("Adding Features - Sentiment")
    df_use = add_feature_sentiment_reviews(df_reviews, df_use)
    print("Adding Features - Pop Zone")
    if train == True:
        global dbscanResults_pop
        global dbscanResults_no_pop
        df_use, dbscanResults_pop, dbscanResults_no_pop = add_features_places_train(df_use)
    else:
        df_use = add_features_places_test(df_use, dbscanResults_pop, dbscanResults_no_pop)
    print("Scaller Latitude and Longitude")
    scaler = preprocessing.MinMaxScaler()
    df_use['latitude'] = scaler.fit_transform(df[['latitude']])
    df_use['longitude'] = scaler.fit_transform(df[['longitude']])
    coluns_drop = ['business_id', 'name', 'address', 'postal_code', 'review_count', 'attributes', 'is_open', 'hours', 'loc', 'categories']
    df_dropped = df_use.drop(coluns_drop, axis=1)
    for i in df_dropped.columns:
        if i not in ['latitude', 'longitude', 'sentiment', 'pop_zone', 'no_pop_zone']:
            df_dropped[i] = pd.Categorical(df_dropped[i])
            df_dropped[i] = df_dropped[i].cat.codes
    return df_dropped

<h2> Chama a função de preparação dos dados e divisão em dados de treino e teste</h2>

In [8]:
df_process = pre_processor(df, df_reviews)
df_train, df_teste = train_test_split(df_process, test_size=0.5, random_state=42)
X = df_train.drop('destaque', axis=1)
y = df_train['destaque']
X_test = df_teste.drop('destaque', axis=1)
y_test = df_teste['destaque']
columns_train = X.columns

Adding Features - Attributes
Adding Features - Categories
Adding Features - Sentiment
Adding Features - Pop Zone
Scaller Latitude and Longitude


<h4> Função para a realização do arredondamento e testes de acertos </h4>
É necessária porque a regressão linear e a rede neural classificam não como 1 ou 0, então criamos uma função para arredondar,
e depois uma função para comparar acertos e erros

In [9]:
###########################################################################################################
## Função que recebe uma lista, e arredonda os valores dela
def round_list(array):
    new_array = []
    for i in array:
        new_array.append(round(i))
    return new_array

###########################################################################################################
## Função que recebe os resultados de um modelo e os dados reais, e realiza a comparação, retornando a porcentagem de acerto
def round_compare(test, real):
    test_round = []
    for i in test:
        test_round.append(round(i))
    correct = 0
    cont = 0
    real_list = list(real)
    while cont < len(real_list):
        if test_round[cont] == real_list[cont]:
            correct += 1
        cont += 1
    return correct/len(real)

<h2> Realiza o treinamento em uma árvore de descisão </h2>

In [10]:
# Instância da árvore de decisão
clf_random_forest = RandomForestClassifier(max_depth = 10, random_state=42)
# Faz o treinamento com os dados de interesse
clf_random_forest.fit(X, y)
result_trees = clf_random_forest.predict(X_test)
clf_random_forest.score(X_test, y_test)

0.8004978220286247

<h2> Realiza o treinamento em uma regressão linear </h2>

In [11]:
linear_regression = LinearRegression()
linear_regression.fit(X,y)
result_regression = linear_regression.predict(X_test)
round_compare(result_regression, y_test)

0.7824517734909769

<h2> Realiza o treinamento em uma rede neural </h2>

In [12]:
mlp = MLPRegressor(hidden_layer_sizes=(15, 5), random_state=1, max_iter=5000, activation='logistic')
mlp_regressor = mlp.fit(X, y)
result_neural = mlp_regressor.predict(X_test)
round_compare(result_neural, y_test)

0.7970130678282514

<h2> Função que realiza o consenso entre os três algoritmos</h2>

In [13]:
def consense_function(r1, r2, r3):
    r1_list = list(r1)
    r2_list = list(r2)
    r3_list = list(r3)
    final_dec = []
    cont = 0
    while cont < len(r1):
        total_votes = r1_list[cont] + r2_list[cont] + r3_list[cont]
        if total_votes > 1:
            final_dec.append(1)
        else:
            final_dec.append(0)
        cont += 1
    return final_dec

In [14]:
consense_result = consense_function(result_trees, result_regression, result_neural)
round_compare(consense_result, y_test)

0.7991288114499067

<h3> Criando o Modelo para ser aplicado na prática </h3>

In [15]:
def model(df, df_reviews):
    ids = df['business_id']
    data = pre_processor(df, df_reviews, train=False)
    
    not_in_model = []
    for i in data.columns:
        if i not in columns_train:
            not_in_model.append(i)
    data =  data.drop(not_in_model, axis=1)
    
    result_trees = clf_random_forest.predict(data)

    result_regression = round_list(linear_regression.predict(data))
    result_neural = round_list(mlp_regressor.predict(data))
    consense_result = consense_function(result_trees, result_regression, result_neural)
    list_tuple = list(zip(ids, consense_result))
    df_final = pd.DataFrame(list_tuple, columns=['business_id', 'destaque'])
    df_final.to_csv("output.txt", header=True, index=False)
    print("The process is finished sucessfully!")
    return df_final

<h1> Colocando o Modelo treinado em prática </h1>

In [16]:
reviews_test_path = os.path.join("data/reviewsTestToronto.csv")
x_test = os.path.join("data/X_testToronto.csv")
df_reviews_test = pd.read_csv(reviews_test_path)
df_test = pd.read_csv(x_test)

<h2> Fazendo a Predição </h2>

In [17]:
df_final = model(df_test, df_reviews_test)

Adding Features - Attributes
Adding Features - Categories
Adding Features - Sentiment
Adding Features - Pop Zone
Scaller Latitude and Longitude
The process is finished sucessfully!
